In [18]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch import tensor
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
from skimage import io, color
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torchmetrics import JaccardIndex, Dice
from tqdm import tqdm
from sklearn.metrics import jaccard_score
import gc
from USSFCNet import USSFCNet

In [3]:
# Load images from folders A and B
folder_A = 'testval/A'
folder_B = 'testval/B'
folder_label = 'testval/label'

In [4]:
device = ("cuda" if torch.cuda.is_available() else "cpu") # Use GPU or CPU for training

In [5]:
print(device)

cuda


In [6]:
# Function to load images from directory
def load_images_from_folder(folder, is_gray = True):
    images = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder,filename)).astype(np.uint8)
        if img is not None:
            if is_gray:
                images.append(color.rgb2gray(img))
            else:    
                images.append(img)
    return images


In [7]:
images_A = load_images_from_folder(folder_A, is_gray=False)

In [8]:
images_B = load_images_from_folder(folder_B, is_gray=False)

In [9]:
labels = load_images_from_folder(folder_label, is_gray = False)

In [10]:
print(len(images_A), len(images_B), len(labels)) 

1200 1200 1200


In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5, 0.5),
])

In [12]:
class ChangeDetectionDataset(Dataset):
    def __init__(self, images_A, images_B, labels):
        self.images_A = images_A
        self.images_B = images_B
        self.labels = labels
#         self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_A = torch.tensor(self.images_A[idx], dtype=torch.float32).permute(2, 0, 1) / 255
        image_B = torch.tensor(self.images_B[idx], dtype=torch.float32).permute(2, 0, 1) / 255
        label = torch.tensor(self.labels[idx], dtype=torch.float32) / 255

#         if self.transform:
#             image_A = self.transform(image_A)
#             image_B = self.transform(image_B)
#             label = self.transform(label)

        return image_A, image_B, label

In [19]:
model = USSFCNet(in_ch=3, out_ch=1, ratio=0.5).to(device)

In [20]:
PATH = 'model_now_2.pickle'
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [21]:
dataset = ChangeDetectionDataset(images_A, images_B, labels)
data_loader = DataLoader(dataset, batch_size=16)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.eval()
val_total_intersections = 0
val_total_unions = 0
with torch.no_grad():
    jaccardArr = []
    for a, b, l in tqdm(data_loader):
        a, b, l = a.to(device), b.to(device), l.to(device)
        outputs = model(a,b)
        outputs = outputs.reshape((-1,l.shape[1],l.shape[2]))
        outputs_binary = (outputs>0.5) * 1
        val_total_intersections += np.logical_and(outputs_binary.cpu().numpy(), l.cpu().numpy()).sum()
        val_total_unions += np.logical_or(outputs_binary.cpu().numpy(), l.cpu().numpy()).sum()
        for i in range(len(outputs_binary)):
            intersection = np.logical_and(outputs_binary[i].cpu().numpy(), l[i].cpu().numpy()).sum()
            union = np.logical_or(outputs_binary[i].cpu().numpy(), l[i].cpu().numpy()).sum()
            if union != 0:
                jaccardArr.append((intersection)/(union))
            else:
                jaccardArr.append(1)
val_jaccard = val_total_intersections / val_total_unions
secondWay = np.array(jaccardArr).mean()
print(f'Validation Jaccard Index : {val_jaccard} | Jaccard Avg : {secondWay} ')

100%|██████████| 75/75 [00:28<00:00,  2.62it/s]


Validation Jaccard Index : 0.039073289064563976 | Jaccard Avg : 0.08604966808773329 
